In [1]:
import torch
assert torch.__version__>='1.2.0', 'Expect PyTorch>=1.2.0 but get {}'.format(torch.__version__)
from torch import nn
import torch.nn.functional as F

import numpy as np

import time
import os
import sys
import pickle

imp_dir = '../Implementations'
sys.path.insert(1, imp_dir)
data_dir = '../Data/criteo'
sys.path.insert(1, data_dir)

import logging
import importlib
importlib.reload(logging)

logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(levelname)-6s %(message)s', level=logging.INFO, datefmt='%H:%M:%S')
logger = logging.getLogger(__name__)

fh = logging.FileHandler('DCN_notebook.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter('%(asctime)s %(levelname)-6s %(message)s'))

logger.addHandler(fh)

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))
torch.cuda.empty_cache()
t = torch.cuda.get_device_properties(DEVICE).total_memory/1024**3
c = torch.cuda.memory_cached(DEVICE)/1024**3
a = torch.cuda.memory_allocated(DEVICE)/1024**3
logger.info('CUDA Memory: Total {:.2f} GB, Cached {:.2f} GB, Allocated {:.2f} GB'.format(t,c,a))

06:02:53 INFO   Device in Use: cuda
06:02:53 INFO   CUDA Memory: Total 11.17 GB, Cached 0.00 GB, Allocated 0.00 GB


## Prepare Data

### Load Dict

In [3]:
embedding_map_dict_pkl_path = os.path.join(data_dir, 'criteo_feature_dict_artifact/categorical_feature_map_dict.pkl')
with open(embedding_map_dict_pkl_path, 'rb') as f:
    embedding_map_dict = pickle.load(f)

### List all available files

In [4]:
np_artifact_dir = os.path.join(data_dir, 'criteo_train_numpy_artifact')
index_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='index', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
value_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='value', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
label_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='label', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))

In [5]:
start = time.time()

train_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[:10]]),
)

logger.info('Training data loaded after {:.2f}s'.format(time.time()-start))

start = time.time()

test_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[10:]]),
)

logger.info('Test data loaded after {:.2f}s'.format(time.time()-start))

06:04:17 INFO   Training data loaded after 78.85s
06:04:27 INFO   Test data loaded after 9.83s


## Create Model

In [6]:
import execution
importlib.reload(execution)
import DCN_BinClf_Torch 
importlib.reload(DCN_BinClf_Torch)

<module 'DCN_BinClf_Torch' from '../Implementations/DCN_BinClf_Torch.py'>

In [7]:
DCN = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)

## Sample Run

In [9]:
cwd = os.getcwd()
checkpoint_dir = os.path.join(cwd, 'DCN_artifact')
checkpoint_prefix = 'DCN'

In [ ]:
execution.train_model_separate_inp(DCN, 
                                   train_data, 
                                   train_data, 
                                   F.binary_cross_entropy_with_logits, 
                                   torch.optim.Adam(DCN.parameters()), 
                                   DEVICE, 
                                   checkpoint_dir, 
                                   checkpoint_prefix 
                                  )

01:20:20 INFO   Epoch 1/5 - Batch 1000/64000 Done - Train Loss: 0.493846, Val Loss: 0.467414
01:24:20 INFO   Epoch 1/5 - Batch 2000/64000 Done - Train Loss: 0.482650, Val Loss: 0.459085
01:28:20 INFO   Epoch 1/5 - Batch 3000/64000 Done - Train Loss: 0.477697, Val Loss: 0.502537
01:32:20 INFO   Epoch 1/5 - Batch 4000/64000 Done - Train Loss: 0.474821, Val Loss: 0.428925
01:36:21 INFO   Epoch 1/5 - Batch 5000/64000 Done - Train Loss: 0.472362, Val Loss: 0.468349
01:40:21 INFO   Epoch 1/5 - Batch 6000/64000 Done - Train Loss: 0.470290, Val Loss: 0.453534
01:44:21 INFO   Epoch 1/5 - Batch 7000/64000 Done - Train Loss: 0.468951, Val Loss: 0.478359
01:48:21 INFO   Epoch 1/5 - Batch 8000/64000 Done - Train Loss: 0.467425, Val Loss: 0.444895
01:52:21 INFO   Epoch 1/5 - Batch 9000/64000 Done - Train Loss: 0.466191, Val Loss: 0.472482
01:56:21 INFO   Epoch 1/5 - Batch 10000/64000 Done - Train Loss: 0.465137, Val Loss: 0.466647
02:00:22 INFO   Epoch 1/5 - Batch 11000/64000 Done - Train Loss: 0.46

## Model Performance

In [10]:
os.listdir(checkpoint_dir)

['DCN_2', 'DCN_4', 'DCN_5', 'DCN_3', 'DCN_1']

In [13]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_1')))
model = model.to(DEVICE)

In [ ]:
logger.info('Model EP1 ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

In [ ]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_2')))
model = model.to(DEVICE)

In [ ]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

In [ ]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_3')))
model = model.to(DEVICE)

In [ ]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

In [ ]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_4')))
model = model.to(DEVICE)

In [ ]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

In [ ]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_5')))
model = model.to(DEVICE)

In [ ]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, test_data, DEVICE)))

## Check Performance on Training Data as Well

In [ ]:
model = DCN_BinClf_Torch.DCN_Layer(len(embedding_map_dict)+60,
                                 20,
                                 26,
                                 13,
                                 [1024 for _ in range(2)],
                                 [0.5 for _ in range(3)],
                                 6).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DCN_1')))
model = model.to(DEVICE)
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score_separate_inp(model, train_data, DEVICE)))